# Aggregate Planning with QUASAR
## Modeling the Decision Problem

Stochastic decision problems in QUASAR can be formulated by using the same type of mathematical expressions as they are common in linear programming. This requires defining a set of decision variables, an objective function, as well as a set of constraints.

We begin by importing the library and creating a *DecisionProblem* that contains our model formulation.

In [19]:
from pyquasar import *

### Variables & Constraints
Variables have to be linked to a decision stage and are defined by addVariable.

Constraints can be formulated using simple algebraic expressions. In the problem at hand the most important constraints link the variables *sell*, *produce* and *inventory*. The corresponding constraint is

$$\text{inventory}_t = \text{inventory}_{t-1} - \text{sell}_t + \text{produce}_t$$

Note that *demand* is the random demand for the product - a specific model for this randomness will be specified below.
### Objective function
With our constraints in place, we finally have to include the objective function, which consists of the costs for buying capacity, the cost for production, the revenue from sales, and cost for maintaining inventory.

$$\max \ \left\{ \sum_{t=1}^{T} \text{sell}_{t} \times \text{price} - \sum_{t=1}^{T} \text{produce}_{t} \times \text{prod_cost} - \text{capacity_cost} \times \text{capacity} - \text{inventory_cost} \times \text{inventory}_t \right\}$$

In [20]:
num_months = 12
capacity_cost = 1
inventory_cost = 0.2
prod_cost = 4
price = 5
init_inventory = 0

model = DecisionProblem()
capacity = model.add_variable(0,"capacity")
model += capacity <= 1000
prev_inventory = init_inventory
for t in range(num_months):
    inventory, produce, sell = model.add_variables(t, "inventory", "produce", "sell")
    model += inventory <= 1000

    #inventory balance
    model += inventory == prev_inventory - sell + produce

    #bounds
    model += sell <= rand("demand")
    model += sell <= prev_inventory
    model += produce <= capacity
    prev_inventory = inventory
    
    # objective function
    model += price*sell - prod_cost*produce - inventory_cost*inventory

# capacity cost in the objective
model += -capacity_cost*capacity

##Modeling the Stochastic Process
Before we can pass the *DecisionProblem* into the optimizer, we have to specify the *MarkovProcess* that drives the evolution of our random variables over time. QUASAR provides a number of classes that allow us to specify different types of stochastic processes.

1. Univariate processes with only one state variable

2. Multivariate processes with multiple state variables

For the sake of simplicity, we choose a simple geometric AR(1) model for the demand. In this model, demand cannot be zero by definition, but is correlated across time, which is often the case in practice.

In [21]:
ar_model = ARModel(name="demand",
                  constant=0.3, 
                  sigma=0.2, 
                  ar_coefficients=[0.8], 
                  initial_state=[5.0], 
                  log_transform=True)
sim = ar_model.simulate(num_stages=num_months, sample_size=100)
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(17,5))
sim.set_time_index(start='2015-01',freq='MS')
sim.demand.fanchart(ax=ax1, ylabel='Million Pieces', title="Annual Demand")
sim.demand.spaghetti(ax=ax2, ylabel='Million Pieces', title="Annual Demand")

## Solving the problem

Putting everything together, we can now generate a lattice and solve the model. The solver runs in a separate thread, so that the notebook is not blocked. To wait for the solver to finish, the *join()* method must be called.

In [22]:
opt = DynamicOptimizer(model, ar_model, num_nodes=50)
opt.solve()
opt.join()

Using the stats *property* of the *opt* object, we can check the status during the solution process.

In [23]:
opt.stats.tail()

,expReward,simReward,stdError,sampleSize,hyperplanes,numSolves,duration
iter,,,,,,,
86,32.32,31.95,1.76,20,17986,47471,33
87,32.32,31.42,1.66,20,18080,48023,33
88,32.32,31.52,1.65,20,18121,48575,33
89,32.32,32.43,1.47,20,18195,49127,34
90,32.31,32.37,0.78,100,18490,49679,35


### Plotting bounds
The ADDP upper bound an the lattice lower bound can be visualized with *DynamicOptimizer.plot()*.

In [24]:
opt.plot()

## Inspecting the solution
To inspect the solution, we simulate the policy on the lattice.

In [25]:
policy = opt.policy
sim = policy.simulate(sample_size = 1000)
sim.set_time_index(start='2015-01',freq='MS')

The result of the simulation is stored in a Pandas dataframe which hows the demand state, the immediate reward, the decision as well as the shadow prices of the time coupling variables. Since this is a Pandas dataframe, analyszing the simulation output is a breeze. 

Let us only take look at the first sample path of our optimized policy.

In [26]:
sim.head(num_months)

rewards decision                            shadow_price           state
                  rewards     sell inventory produce capacity    inventory capacity demand
series stage                                                                              
0      2015-01-01  -30.35     0.00      5.75    5.75     6.22         4.27     0.97   5.00
       2015-02-01    3.96     5.34      5.80    5.40     6.22         4.96     0.95   5.34
       2015-03-01    4.77     4.85      5.54    4.59     6.22         4.14     0.72   4.85
       2015-04-01    4.70     4.41      5.21    4.07     6.22         3.78     0.01   4.41
       2015-05-01   -0.07     5.21      6.22    6.22     6.22         4.83     0.85   6.02
       2015-06-01    7.27     4.08      5.16    3.02     6.22         4.59     0.20   4.08
       2015-07-01   -0.31     5.16      6.22    6.22     6.22         4.49     0.37   5.46
       2015-08-01    2.86     5.72      6.60    6.11     6.22         4.13     0.19   5.72
       2015-09-01    6.90     6.60      6.22    6.22     6.22         4.75     0.21   6.69
       2015-10-01    3.24     5.88      6.55    6.22     6.22         5.00     0.80   5.88
       2015-11-01   14.49     5.35      4.06    2.87     6.22         4.80     0.00   5.35
       2015-12-01   20.32     4.06      0.00    0.00      nan          nan      nan   7.34

### The *true* profit distribution
The simulated profit of the policy under the AR demand model may deviate from the simulated lower bound if not enough scenarios have been used to construct the lattice. Let us therefore take a look at the reward distribution.

In [27]:
sim.rewards.groupby_series.sum().describe().transpose()

,count,mean,std,min,25%,50%,75%,max
rewards,"1,000.00",32.51,7.24,12.60,27.10,32.95,37.95,47.92


### Getting first stage decision
The production capacity is a first stage decision variable. Hence, it is independent of the scenarios and only has one deterministic value for all scenarios.

In [28]:
policy.first_stage_solution()

{
  "decisions": {
    "capacity": 6.22, 
    "inventory": 5.75, 
    "produce": 5.75, 
    "sell": 0.00
  }, 
  "rewards": -30.35, 
  "shadow_prices": {
    "capacity": 0.14, 
    "inventory": 3.88
  }
}

### Plotting stochastic decision variables
The variables *inventory*, *produce*, and *sell* vary with different scenarios and are therefore random. We use a fanplot to visualize this randomness over time.

In [29]:
f, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(16,5))
sim.decision['inventory'].fanchart(ax1, ylabel='Million Pieces', title="Annual Inventories")
sim.decision['produce'].fanchart(ax2, ylabel='Million Pieces', title="Annual Production Quantities")

### Service Level
The service level is the percentage of demand that can be fulfilled from stock. The $\alpha$-service is event-oriented counting the percentage of in-stock instances, where as the $\beta$-service is quantity-oriented accumulating the percentage of demand served from stock.

In [30]:
alpha = np.sign(sim.state['demand']-sim.decision['inventory']).clip(0).mean()
beta = 1-(sim.state['demand']-sim.decision['inventory']).sum()/sim.state['demand'].sum()
print("%s-Service = %.3f"%(u"\u03B1",alpha))
print("%s-Service = %.3f"%(u"\u03B2",beta))

α-Service = 0.306
β-Service = 0.975


### Exporting the simulation result
We can easily export the results from simulating the optimal policy into a CSV for further usage elsewhere.

In [31]:
sim.to_csv('agg_model.csv')